# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
from azureml.core import Environment, ScriptRunConfig
import os
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
import logging
import csv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
import joblib
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core import Model

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.get(name = 'quick-starts-ws-260426',
                   subscription_id = '61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30',
                   resource_group = 'aml-quickstarts-260426')

experiment_name = 'hyper-heart'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-260426
Azure region: westus2
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-260426


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [8]:
cluster_name = "notebook260426"

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, delay_evaluation= 5, evaluation_interval=1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
{
    '--C': loguniform(-3,3),
    '--max_iter': choice(100, 200, 300, 400)
})

#env = Environment.get(ws, name='MyEnvironment')
sklearn_env = Environment.from_conda_specification(name='sklearn-1.0:37', file_path='conda_dependencies.yml')

#TODO: Create your estimator and hyperdrive config
"""estimator = SKLearn(source_directory= './',
                    entry_script= "train.py",
                    compute_target= cluster_name)"""
src = ScriptRunConfig(
    source_directory= ".",
    script = "train.py",
    compute_target= cluster_name,
    environment = sklearn_env
)

hyperdrive_run_config = HyperDriveConfig(
    run_config= src,
    hyperparameter_sampling=param_sampling,
    policy= early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=4,
    max_concurrent_runs=4
)

In [9]:
#TODO: Submit your experiment
hdr = experiment.submit(config = hyperdrive_run_config)

RunDetails(hdr).show()

notebook_run = experiment.start_logging()

notebook_run.log(name="message", value = "Hello from run!")

print(notebook_run.get_status())

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Running


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

